In [168]:
import subprocess
import os
import argparse
import pandas as pd
import numpy as np
import requests
from sqlalchemy import create_engine
from dotenv import load_dotenv
import logging
from sqlalchemy import text
import duckdb
import datetime
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)

load_dotenv()
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(DATABASE_URL)

In [169]:
engine 
start_year=2024 
end_year=2024 
parquet_folder = 'data/raw/nflfastr/'

In [170]:
weekly_df = pd.read_sql("SELECT * FROM weekly_stats", engine)
players_df = pd.read_sql("SELECT * FROM players", engine)
teams_df = pd.read_sql('SELECT * FROM teams', engine)
games_df = pd.read_sql('SELECT * FROM games', engine)
depth_df = pd.read_sql('SELECT * FROM depth_chart', engine)
injuries_df = pd.read_sql('SELECT * FROM injuries', engine)
schedule_df = pd.read_sql("SELECT season, week, stadium, game_date FROM games", engine)
base_features_df = pd.read_sql("SELECT * FROM player_weekly_features", engine)
pre_features_df = pd.read_sql("SELECT * FROM player_weekly_features", engine)
final_df = pd.read_sql("SELECT * FROM features", engine)
qb_df = pd.read_sql("SELECT * FROM qb_features", engine)
rb_df = pd.read_sql("SELECT * FROM rb_features", engine)
wr_df = pd.read_sql("SELECT * FROM wr_features", engine)
te_df = pd.read_sql("SELECT * FROM te_features", engine)

In [259]:
games_df

,season,week,home_team,away_team,stadium,game_date,home_team_id,away_team_id
0,2022,1,LAR,BUF,SoFi Stadium,2022-09-09,18.0,3.0
1,2022,1,ATL,NO,Mercedes-Benz Stadium,2022-09-11,1.0,22.0
2,2022,1,CHI,SF,Soldier Field,2022-09-11,5.0,27.0
3,2022,1,CIN,PIT,Paycor Stadium,2022-09-11,6.0,26.0
4,2022,1,DET,PHI,Ford Field,2022-09-11,10.0,25.0
...,...,...,...,...,...,...,...,...
4343,2025,18,PIT,BAL,Acrisure Stadium,2026-01-04,26.0,2.0
4344,2025,18,SF,SEA,Levi's Stadium,2026-01-04,27.0,28.0
4345,2025,18,TB,CAR,Raymond James Stadium,2026-01-04,29.0,4.0
4346,2025,18,JAX,TEN,EverBank Stadium,2026-01-04,14.0,30.0


In [262]:
from datetime import datetime, timezone

today = datetime.now(timezone.utc)
games = pd.read_sql('SELECT * FROM games', engine)
games["game_date"] = pd.to_datetime(games["game_date"], utc=True)

upcoming_games = games[games["game_date"] >= today]
if not upcoming_games.empty:
    next_week = int(upcoming_games.sort_values("game_date").iloc[0]["week"])
    print(next_week)
else:
    # All games passed; return max week + 1
    print(int(games["week"].max()) + 1)

1


In [263]:
upcoming_games

,season,week,home_team,away_team,stadium,game_date,home_team_id,away_team_id
1358,2025,1,PHI,DAL,Lincoln Financial Field,2025-09-05 00:00:00+00:00,25.0,8.0
1359,2025,1,LAC,KC,Corinthians Arena,2025-09-06 00:00:00+00:00,17.0,15.0
1360,2025,1,ATL,TB,Mercedes-Benz Stadium,2025-09-07 00:00:00+00:00,1.0,29.0
1361,2025,1,CLE,CIN,Huntington Bank Field,2025-09-07 00:00:00+00:00,7.0,6.0
1362,2025,1,IND,MIA,Lucas Oil Stadium,2025-09-07 00:00:00+00:00,13.0,19.0
...,...,...,...,...,...,...,...,...
4343,2025,18,PIT,BAL,Acrisure Stadium,2026-01-04 00:00:00+00:00,26.0,2.0
4344,2025,18,SF,SEA,Levi's Stadium,2026-01-04 00:00:00+00:00,27.0,28.0
4345,2025,18,TB,CAR,Raymond James Stadium,2026-01-04 00:00:00+00:00,29.0,4.0
4346,2025,18,JAX,TEN,EverBank Stadium,2026-01-04 00:00:00+00:00,14.0,30.0


In [266]:
datetime.now().year

2025